# File Description
The file “2014-2018 PM10 LisAvLib” contains a time series of hourly-levels of $PM_{10}$ particles (in micrograms per cubic meter), collected at Avenida da Liberdade monitoring station in Lisbon from 01/01/2014 to 31/12/2018.

# Main Goal
Fit a **SARIMA-type** model to the time series representing 24-h average levels of $PM_{10}$ particles. 

# Process

1. Libraries;
2. Dataset Importation;
3. Functions;
4. Exploratory Data Analysis:
    1. Handling Missing Values;
    2. Statistical and Empirical Analysis;
    3. ACF and PACF;
    4. Decomposing;
    5. Identification of the dependence of orders and degree of differencing;
    6. Transformations.
5. Model Fitting and Diagnostics:
    1. Parameter Estimation;
    2. Residual diagnostics and model selection.
6. Cross-validation;
7. Forecast:
    1. Forecast the data into the future up to 5 time periods ahead;
    2. Calculate 95% prediction intervals for each of the 5 forecasts.

# 1. Libraries

In [ ]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
import xlrd
warnings.filterwarnings("ignore")

import pandas as pd
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from pandas import Series

import statsmodels.api as sm
import statsmodels.tsa.api as smt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.api import qqplot

from pmdarima.arima import auto_arima
import pmdarima.arima as pm

import sklearn
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, mean_squared_log_error

from scipy.interpolate import interp1d
import scipy.stats as stats
from scipy.stats import boxcox

import pylab
import matplotlib
import seaborn as sns
from matplotlib import pyplot
from numpy import sqrt

In [ ]:
matplotlib.rcdefaults()

matplotlib.rcParams['figure.figsize'] = (12, 4)

font = {'family' : 'monospace',
        'size'   : 16}

matplotlib.rc('font', **font)

matplotlib.style.use('seaborn-whitegrid')

tfont = {'family' : 'monospace',
        'weight' : 'bold',
        'size'   : 18}

yfont = {'family' : 'serif',
        'weight' : 'bold',
        'size'   : 18}

lfont = {'family' : 'serif',
        'weight' : 'bold',
        'size'   : 14}

# 2. Import dataset

- The name of the 2 initial columns was changes to 'Date' and 'PM_10' ir order to facilitate future analysis.

- Set the first column, 'Date', as the index of the dataframe.

In [ ]:
df= pd.read_excel('2014-2018 PM10 LisAvLib.xlsx', header=0)

In [ ]:
df.columns = ['Date','PM_10']
df=df.set_index('Date')

# 3. Functions

- The functions below will be used along the course of this project. They were created in order to facilitate the user experience.

In [ ]:
def knn_mean(ts, n):
    out = np.copy(ts)
    for i, val in enumerate(ts):
        if np.isnan(val):
            n_by_2 = np.ceil(n/2)
            lower = np.max([0, int(i-n_by_2)])
            upper = np.min([len(ts)+1, int(i+n_by_2)])
            ts_near = np.concatenate([ts[lower:i], ts[i:upper]])
            out[i] = np.nanmean(ts_near)
    return out

In [ ]:
def imputation(x, n_rep, n_na, n_k):

    #Inputs:
        #x - 24h dataframe
        #n_rep - number of cycles
        #n_na - number of missing values on the new test dataframe
        #n_k - number of k neighbours
        
    warnings.filterwarnings("ignore") # specify to ignore warning messages
    
    #Creating a dataframe with the first 500 values of the 24-hour average dataset
    y=pd.DataFrame(x,columns='PM_10'.split(),index=x.index[:500])
    
    #Creating 5 different arrays to store the errors for each imputation technique on each cycle
    c1=[]
    c2=[]
    c3=[]
    c4=[]
    c5=[]

    print('Forward Fill | Backward Fill | Linear Interpolation | Cubic Interpolation | k-NN')
    for i in range(0, n_rep):
        #Create a new column on the dfIMPtest dataframe with the same values as the column PM_10
        y = y.assign(PM_10NA=y['PM_10'])
        
        #Replace the values of 10 random objects from the column 'PM_10' with NA value
        change = y.sample(n_na).index
        y.loc[change,'PM_10NA'] = np.nan
        
        #Forward Fill
        y['ffill'] = y['PM_10NA'].ffill()
        error_ffill = np.round(mean_squared_error(y['PM_10'], y['ffill']), 2)
        c1.append(error_ffill)
        
        #Backward Fill
        y['bfill'] = y['PM_10NA'].bfill()
        error_bfill = np.round(mean_squared_error(y['PM_10'], y['bfill']), 2)
        c2.append(error_bfill)
        
        #Creating a new column of y dataframe which will be used in Linear and Cubic Interpolation
        y['rownum'] = np.arange(y.shape[0])
        y_nona = y.dropna(subset = ['PM_10NA'])

        #Linear Interpolation
        f = interp1d(y_nona['rownum'], y_nona['PM_10NA'])
        y['linear_fill'] = f(y['rownum'])
        error_lininter = np.round(mean_squared_error(y['PM_10'], y['linear_fill']), 2)
        c3.append(error_lininter)
        
        #Cubic Interpolation
        f2 = interp1d(y_nona['rownum'], y_nona['PM_10NA'],kind='cubic')
        y['cubic_fill'] = f2(y['rownum'])
        error_cubicinter = np.round(mean_squared_error(y['PM_10'], y['cubic_fill']), 2)
        c4.append(error_cubicinter)
        
        #k-NN
        y['knn_mean'] = knn_mean(y.PM_10NA.values, n_k)
        error_knn = np.round(mean_squared_error(y['PM_10'], y['knn_mean']), 2)
        c5.append(error_knn)
        
        print('{} - {} | {} | {} | {} | {} '.format(i+1, c1[i],c2[i],c3[i],c4[i],c5[i]))
        #print('Ite:',i)
    
    #Average error values 
    avg_forward=np.mean(c1)
    avg_backward=np.mean(c2)
    avg_lininter=np.mean(c3)
    avg_cubicinter=np.mean(c4)
    avg_knn=np.mean(c5)
    
    print('Forward Fill Average:',avg_forward)
    print('Backward Fill Average:',avg_backward)
    print('Linear Interpolation Average:', avg_lininter)
    print('Cubic Interpolation Average:', avg_cubicinter)
    print('k-NN Average:', avg_knn)

In [ ]:
def dataframe_analysis(df,nan):
    print('----Descriptive-Analysis----')
    print('Description: ',df.describe())
    print('----------------------------')
    print('Mean : ',df.mean())
    print('Variance : ',df.var())
    print('Skewness : ',df.skew())
    print('Kurtosis : ',df.kurt())
    print('--------------@-------------')
    
    df.plot(figsize=(12, 4))
    if nan==0:
        fig, axes = plt.subplots(nrows=1, ncols=2)
        df.plot.kde(figsize=(12, 4),ax=axes[0])
        qqplot(df, line='q', fit=True, ax=axes[1])
        plt.show()

In [ ]:
def ARMA_search(y,pr,qr):

    #Inputs:
        #y - Dataframe
        #p_r - max range of p
        #q_r - max range of q
        
    warnings.filterwarnings("ignore") # specify to ignore warning messages

    # Define the p and q parameters to take any value between 0 and p_r/q_r, respectively.
    p = range(0, pr)
    q = range(0, qr)
    
    # Generate all different combinations of p and q
    pq = list(itertools.product(p,q))

    c1=[]
    c2=[]
    i=0
    
    for k in pq:
        try:
            model=sm.tsa.ARMA(y,order=k)
            results = model.fit()
                
            c1.append(results.aic)
            c2.append(results.bic)
            print('{}-ARMA{} - AIC:{} | BIC:{}'.format(i,k, results.aic, results.bic))
            i=i+1
        except:
            continue
  
    index_min1 = np.argmin(c1)
    index_min2 = np.argmin(c2)

    print('Valor minimo de AIC:',np.min(c1))
    print('Combinacao de parametros:', index_min1)
    
    print('Valor minimo de BIC:',np.min(c2))
    print('Combinacao de parametros:', index_min2)      

In [ ]:
def cross_validation_results(y,train,pr,qr):
    #Inputs:
        #y - Dataframe
        #p_r - max range of p
        #q_r - max range of q
        
    warnings.filterwarnings("ignore") # specify to ignore warning messages

    start_index = y.index.min()
    end_index = y.index.max()
    # Define the p and q parameters to take any value between 0 and p_r/q_r, respectively.
    p = range(0, pr)
    q = range(0, qr)
    
    # Generate all different combinations of p and q
    pq = list(itertools.product(p,q))
    
    columns = ['ARMA','r2_score','mae','mse','msle','rmse']
    rows = []
    for k in pq:
        try:
            model=sm.tsa.ARMA(train,order=k)
            results = model.fit(method='css-mle')
                
            pred = results.predict(start=start_index, end=end_index, dynamic=True)    
            
            #mean_absolute_percentage_error = np.mean(np.abs((y - pred) / y)) * 100
            #,mean_absolute_percentage_error)
            #,'mape'
            
            row=[k,r2_score(y, pred),mean_absolute_error(y, pred),mean_squared_error(y, pred), mean_squared_log_error(y, pred), np.sqrt(mean_squared_error(y, pred))]
            rows.append(row)
            
        except:
            continue
    df = pd.DataFrame(rows, columns=columns)
    return df


In [ ]:
#Plot comparing prediction and actual values
#Input: (dfDtransf.loc['data(starting point)':], Train dataframe, AR order, MA order)
#In this case, the AR and MA order in the input is the actual order.
def cross_validation_plot(y, train, test, p, q):
    warnings.filterwarnings("ignore") # specify to ignore warning messages

    start_index = test.index.min()
    end_index = test.index.max()
    
    model=sm.tsa.ARMA(train,order=(p,q))
    results = model.fit(method='css-mle')
                
    pred = results.predict(start=start_index, end=end_index, dynamic=True) 
    
    fig, ax = plt.subplots(figsize=(12, 8))
    y.plot(ax=ax)
    pred.plot(ax=ax);
    ax.legend(["Observed", "Forecast"]);

# 4. Exploratory Data Analysis

## 4.1 Initial Data Analysis

In [ ]:
#Total number of 1-hour PM_10 Particles. It also counts the NA.
df.shape

In [ ]:
#Number of missing PM_10 values in the whole dataframe.
df.isnull().sum()

In [ ]:
#Second attribute = 1, since there are NA values on the dataframe.
dataframe_analysis(df,1)

## 4.2 Resampling the Initial Data

- As it was asked and in order to have an easier understanding of the data, it was created a **24-hour average** dataframe of the 1-hour $PM_{10}$ particle set.

Creating the desired 24-hour average dataframe.

In [ ]:
df24 = df.resample('D').mean()

**General information** about the initial dataframe

In [ ]:
#Total number of 1-hour PM_10 Particles. It also counts the NA.
df24.shape

In [ ]:
#Number of missing PM_10 values in the whole dataframe.
df24.isnull().sum()

In [ ]:
#Second attribute = 1, since there are NA values on the dataframe.
dataframe_analysis(df24,1)

## 4.3 Handling Missing Values

### 4.3.1 Visual Analysis


Making a new dataframe with only the **NAs objects**

In [ ]:
dfNA = df24.copy()

In [ ]:
dfNA.shape

In [ ]:
dfNA.isnull().sum()

Create an **array** with the values of $PM_{10}$ particles

In [ ]:
pm10 = dfNA['PM_10'].values

Replace the NAs values with '1' and the others values with '0' in order analyze where the NAs are placed in the 5 year timeframe

In [ ]:
for i in np.nditer(pm10,op_flags=['readwrite']):
    if np.isnan(i):
        i[...]=1
    else:
        i[...]=0

- Creating a dataframe with the binary values of pm10

- Replace the values of the column PM_10 from the dfNA dataframe with the values available in the pm10a dataframe

In [ ]:
pm10a = pd.DataFrame(pm10)

dfNA = dfNA.assign(PM_10=pm10a.values)

Creating a new column on dfNA dataframe with **trimester intervals** from *2014 till the end of 2018*

In [ ]:
dfNA['Interval [Trimester]'] = pd.cut(dfNA.index,20,
                        labels=['(2014-01-01,2014-04-02)','(2014-04-03,2014-07-02)',
                                '(2014-07-03,2014-10-01)','(2014-10-02,2015-01-01)',
                                '(2015-01-02,2015-04-02)','(2015-04-03,2015-07-02)',
                                '(2015-07-03,2015-10-01)','(2015-10-02,2016-01-01)',
                                '(2016-01-02,2016-04-01)','(2016-04-02,2016-07-01)',
                                '(2016-07-02,2016-09-30)','(2016-10-01,2017-12-31)',
                                '(2017-01-01,2017-04-01)','(2017-04-02,2017-07-01)',
                                '(2017-07-02,2017-09-30)','(2017-10-01,2018-12-31)',
                                '(2018-01-01,2018-04-01)','(2018-04-02,2018-07-01)',
                                '(2018-07-02,2018-09-30)','(2018-10-01,2018-12-31)'])

In [ ]:
dfNA['Month'] = pd.cut(dfNA.index, 60)

Create a new dataframe of dfNA where the values of $PM_{10}$ particles are grouped up by it's correspondent trimester

In [ ]:
new_dfNA = dfNA[['Interval [Trimester]','PM_10']].groupby('Interval [Trimester]').sum()

In [ ]:
new_dfNA1 = dfNA[['Month','PM_10']].groupby(dfNA.index.month).sum()

In [ ]:
new_dfNA.rename(columns={'PM_10':'NA'},inplace=True)

In [ ]:
new_dfNA1.rename(columns={'PM_10':'NA'},inplace=True)

Bar plot of the dataframe new.dfNA on which are able to see the number of days with N/A values in each trimester. 

In [ ]:
new_dfNA.plot(figsize=(8, 6),kind='barh')
plt.title('Number of days with NA values',**tfont)
plt.ylabel('Trimester', **yfont)

In [ ]:
new_dfNA1.plot(figsize=(10, 6),kind='barh')
plt.title('Number of days with NA values',**tfont)
plt.ylabel('Month', **yfont)

**Monthly** values of particles in order to see if there is any correlation between the time when there is a NA value and the actual PM_10 particle value at the same time. (time in month).

In [ ]:
def monthly_correlation(df1,df2):
    import plotly.io as pio
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    pio.templates.default = "none"
    
    dfM = df1.resample('M').mean()
    
    dfM['Month'] = dfM.index
    
    new_dfM = dfM[['Month','PM_10']].groupby(dfM.index.month).mean()
    
    new_dfM['Month NA'] = df2
    
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Bar(x=months, y=new_dfM['PM_10'], name='Average PM_10 particles per month'),
        secondary_y=False,
    )

    fig.add_trace(
        go.Bar(x=months, y=new_dfM['Month NA'], name="Number of Missig Values per month"),
        secondary_y=True,
    )
    
    # Add figure title
    fig.update_layout(
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        legend=dict(
            x=0,
            y=1,
            traceorder="normal",
            font=dict(
                family="sans-serif",
                size=12,
                color="black"
            )#,
            #bgcolor="LightSteelBlue",
            #bordercolor="Black",
            #borderwidth=1
        ),
        barmode='group',
        bargap=0.4, # gap between bars of adjacent location coordinates.
        bargroupgap=0.2 # gap between bars of the same location coordinate.
    )

    # Set x-axis title
    fig.update_xaxes(title_text="<b>Month<b>")

    # Set y-axes titles
    fig.update_yaxes(title_text=r"$PM_{10}$", secondary_y=False)
    fig.update_yaxes(title_text="<b>NAs<b>", secondary_y=True, range=[0,12])

    fig.show()
    
    dfM

In [ ]:
monthly_correlation(df,new_dfNA1)

### 4.3.2 Imputation

#### 4.3.2.1 Choosing the imputation method

In [ ]:
#Inputs:
        #1 - 24h dataframe
        #2 - n_rep - number of cycles
        #3 - n_na - number of missing values on the new test dataframe
        #4 - n_k - number of k neighbours
imputation(df24,21, 2, 5)

#### 4.3.2.2 Applying the chosen imputation method - Linear Interpolation 

In [ ]:
df24['rownum'] = np.arange(df24.shape[0])

df24_nona = df24.dropna(subset = ['PM_10'])

f = interp1d(df24_nona['rownum'], df24_nona['PM_10'])

df24['PM_10_F'] = f(df24['rownum'])

In [ ]:
dfD = df24.copy()

In [ ]:
dfD.drop(['PM_10','rownum'], axis=1, inplace=True)

In [ ]:
dfD.rename(columns={'PM_10_F':'PM_10'},inplace=True)

## 4.4 Dataframe Split in two parts

- dfD_train contains all the observations till 26-12-2018;
- dfD_test constains the last 5 observations (26-12-2018 till 31-12-2018);

In [ ]:
dfD_train=dfD[:'2018-12-26'].copy()

In [ ]:
dfD_test=dfD['2018-12-27':].copy()

## 4.5 Statistical/Empirical Summary

- Descriptive statistics
- Skewness
- Kurtosis
- Histogram
- QQ-plot

In [ ]:
dataframe_analysis(dfD_train['PM_10'],0)

## 4.6 Time Series,  ACF and PACF

In [ ]:
dfD_train.plot(figsize=(12, 4))
pyplot.xlabel("Date",**yfont)
pyplot.ylabel("$PM_{10}$", **yfont)
pyplot.legend("")
pyplot.title("")

In [ ]:
pyplot.figure(figsize=(12,4))
plot_acf(dfD_train,ax=pyplot.gca(), lags=40)
pyplot.xlabel("Lag",**yfont)
pyplot.ylabel("ACF", **yfont)
pyplot.title("")
pyplot.show()

In [ ]:
pyplot.figure(figsize=(12,4))
plot_pacf(dfD_train,ax=pyplot.gca(), lags=40)
pyplot.xlabel("Lag",**yfont)
pyplot.ylabel("PACF", **yfont)
pyplot.title("")
pyplot.show()

## 4.7 Decomposing

- STL function: https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.STL.html
- STL.fit function: https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.STL.fit.html#statsmodels.tsa.seasonal.STL.fit

- seasonal_decompose function: https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.seasonal_decompose.html

In [ ]:
matplotlib.rcParams['figure.figsize'] = (25, 15)

plt.figure()
decomp=STL(dfD_train)
res = decomp.fit()
resp=res.plot()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (25, 15)
decomp=seasonal_decompose(dfD_train,model='additive')
decomp.plot()
pyplot.show()

## 4.8 Differencing

- It will be tested if there is a need for ARIMA differencing and SEASONAL differencing by using the pmdarima library.

- ARIMA differencing function: https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.ndiffs.html

- SEASONAL differencing function: https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.nsdiffs.html

In [ ]:
n_diffs = pm.ndiffs(dfD_train, max_d=10)
print("n_diffs:", n_diffs)

In [ ]:
n_nsdiffs = pm.nsdiffs(dfD_train,5, max_D=10)
print("n_nsdiffs:", n_nsdiffs)

# 4.9 Transformation

### 4.9.1 Boxcox Function

- Box-cox transformation function: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.boxcox.html

- lambda = -1. is a reciprocal transform;
- lambda = -0.5 is a reciprocal square root transform;
- lambda = 0.0 is a log transform;
- lambda = 0.5 is a square root transform;
- lambda = 1.0 is no transform.

In [ ]:
dfD_trainT = dfD_train.copy()

In [ ]:
a = boxcox(dfD_trainT['PM_10'].values)
print('array:',a)
#best lmbda = 0.02070343099348869

In [ ]:
dfD_trainT['PM_10'] = boxcox(dfD_trainT['PM_10'], lmbda=0)

Transforming both dfD and the test dataframe

In [ ]:
dfD_T = dfD.copy()

In [ ]:
dfD_T['PM_10'] = boxcox(dfD['PM_10'], lmbda=0)

In [ ]:
dfD_testT = dfD_test.copy()

In [ ]:
dfD_testT['PM_10'] = boxcox(dfD_test['PM_10'], lmbda=0)

### 4.9.2 KDE and QQ plot

In [ ]:
dfD_trainT['PM_10'].plot.kde(figsize=(12, 4))
plt.xlim((1.7,5.4))
#plt.xlim((-5,105))
#plt.xlim((1.5,4.5))

pyplot.xlabel("PM_10",**yfont)
pyplot.ylabel("Density", **yfont)
pyplot.legend("")
pyplot.title("")

In [ ]:
#fig, axes = plt.subplots(nrows=1, ncols=1)
qqplot(dfD_trainT['PM_10'], line='q', fit=True)
pyplot.xlabel("Theoretical Quantiles",**yfont)
pyplot.ylabel("Sample Quantiles", **yfont)
pyplot.legend("")
pyplot.title("")
plt.show()

In [ ]:
dataframe_analysis(dfD_trainT['PM_10'],0)

### 4.9.3 Time Series, ACF and PACF

In [ ]:
dfD_trainT.plot(figsize=(12, 4))
pyplot.xlabel("Date",**yfont)
pyplot.ylabel("$log(PM_{10})$", **yfont)
pyplot.legend("")
pyplot.title("")

In [ ]:
pyplot.figure(figsize=(12,4))
plot_acf(dfD_trainT,ax=pyplot.gca(), lags=40)
pyplot.xlabel("Lag",**yfont)
pyplot.ylabel("ACF", **yfont)
pyplot.title("")
pyplot.show()

In [ ]:
pyplot.figure(figsize=(12,4))
plot_pacf(dfD_trainT,ax=pyplot.gca(), lags=40)
pyplot.xlabel("Lag",**yfont)
pyplot.ylabel("PACF", **yfont)
pyplot.title("")
pyplot.show()

### 4.9.4 Decomposing

- STL function: https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.STL.html
- STL.fit function: https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.STL.fit.html#statsmodels.tsa.seasonal.STL.fit

- seasonal_decompose function: https://www.statsmodels.org/dev/generated/statsmodels.tsa.seasonal.seasonal_decompose.html

In [ ]:
matplotlib.rcParams['figure.figsize'] = (25, 15)

plt.figure()
decomp=STL(dfD_trainT).fit()
resp=decomp.plot()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (25, 15)
decomp=seasonal_decompose(dfD_trainT,model='additive')
decomp.plot()
pyplot.show()

# 5. Model Fitting and Diagnostics

## 5.1 Model Fitting

- arma function: https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARMA.html
- auto_arima function: https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.auto_arima.html

In [ ]:
#Inputs:
        #y - Dataframe
        #p_r - max range of p
        #q_r - max range of q
ARMA_search(dfD_trainT,5,5)

In [ ]:
modl = auto_arima(dfD_trainT, start_p=1, start_q=1, max_p=10, max_q=10, seasonal=False, d=0, D=0, 
                     stepwise=True, suppress_warnings=True, error_action='ignore')

print("model:", modl)

## 5.2 Residual Diagnostics

### 5.2.1 Residuals Summary

In [ ]:
model = sm.tsa.ARMA(dfD_trainT, order=(1,1))
results = model.fit(method='css-mle')
print(results.summary())

In [ ]:
aicc = pm.ARIMA(order=(1, 0, 1)).fit(dfD_trainT).aicc()
print('AICc:',aicc)

In [ ]:
dataframe_analysis(results.resid,0)

### 5.2.2 Residuals Time Series, KDE, QQ, ACF and PACF plots

In [ ]:
results.resid.plot(figsize=(12, 4))
pyplot.xlabel("Date",**yfont)
pyplot.ylabel("$log(PM_{10}).resid$", **yfont)
pyplot.legend("")
pyplot.title("")

In [ ]:
results.resid.plot.kde(figsize=(12, 4))
pyplot.xlabel("PM_10",**yfont)
pyplot.ylabel("Density", **yfont)
pyplot.legend("")
pyplot.title("")

In [ ]:
qqplot(results.resid, line='q', fit=True, loc=0, scale=1)
pyplot.xlabel("Theoretical Quantiles",**yfont)
pyplot.ylabel("Sample Quantiles", **yfont)
pyplot.legend("")
pyplot.title("")
plt.show()

In [ ]:
pyplot.figure(figsize=(12,4))
plot_acf(results.resid,ax=pyplot.gca(), lags=20)
pyplot.xlabel("Lag",**yfont)
pyplot.ylabel("ACF", **yfont)
pyplot.title("")
pyplot.show()

In [ ]:
pyplot.figure(figsize=(12,4))
plot_pacf(results.resid,ax=pyplot.gca(), lags=20)
pyplot.xlabel("Lag",**yfont)
pyplot.ylabel("PACF", **yfont)
pyplot.title("")
pyplot.show()

## 5.3 Tests of Normality and Independence

### 5.3.1 Shapiro Wilk (Test of Normality)

- Function: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html

- **Output**: Shapiro-Wilk value; p-value;

In [ ]:
stats.shapiro(results.resid)

### 5.3.2 Jarque Bera Test (Test of Normality)

- Function: https://www.statsmodels.org/devel/generated/statsmodels.stats.stattools.jarque_bera.html

- **Output**: Jarque-Bera; p-value; estimated skewness; estimated kurtosis

In [ ]:
sm.stats.jarque_bera(results.resid)

### 5.3.3 Ljung-Cox Test (Test of Normality)
- Function: https://www.statsmodels.org/dev/generated/statsmodels.stats.diagnostic.acorr_ljungbox.html

- **Output**: Ljung-Box value; p-value;

In [ ]:
sm.stats.acorr_ljungbox(results.resid,return_df=True, lags=10,model_df=3)

# 6. Cross-Validation

In [ ]:
# Input: (Test dataframe, Train dataframe, max AR order, max MA order)
# It starts from 0, then if you want a max order of 3 for AR and MA, you need an input of 4 in both parameters.
cross_validation_results(dfD_testT, dfD_trainT, 4,4)

In [ ]:
#Plot comparing prediction and actual values

#Input: (dfDtransf.loc['data(starting point)':], Train dataframe, Test dataframe, AR order, MA order)
#In this case, the AR and MA order in the input is the actual order.
cross_validation_plot(dfD_T.loc['2018-12-27':],dfD_trainT, dfD_testT,1,0)

# 9. Forecast

In [ ]:
#Input: (dataframe, order)
model = sm.tsa.ARMA(dfD_T, order=(1,1))
results = model.fit(method='css-mle')

In [ ]:
fig, ax = plt.subplots(figsize=(18,6))
fig=results.plot_predict(start='2018-12-23', end='2019-01-05', ax=ax)
legend = ax.legend(["Observed", "Forecast","95% Conf. Interval"], loc='upper left')
ax.set_xlabel("Date",**yfont)
ax.set_ylabel("$log(PM_{10})$", **yfont)

In [ ]:
pred = results.predict(start='2019-01-01', end='2019-01-05', dynamic=True)    

In [ ]:
pred

Arrays of the function below:

- forecastndarray
    - Array of out of sample forecasts

- stderrndarray
    - Array of the standard error of the forecasts.

- conf_intndarray
    - 2d array of the confidence interval for the forecast



In [ ]:
results.forecast(steps=5, alpha=.05)